In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/cornell_movie_dialogs_corpus.zip" -d "/content/cornell_movie"

Archive:  /content/drive/MyDrive/cornell_movie_dialogs_corpus.zip
   creating: /content/cornell_movie/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/.DS_Store  
   creating: /content/cornell_movie/__MACOSX/
   creating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/chameleons.pdf  
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: /content/cornell_movie/cornell

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import re
import time
import os
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [ ]:
import codecs

with codecs.open("/content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    conversations = []
    for line in lines:
        data = line.split(" +++$+++ ")
        conversations.append(data)

In [ ]:
chats = {}
for tokens in conversations:
    if len(tokens) > 4:
        idx = tokens[0][1:]
        chat = tokens[4]
        chats[int(idx)] = chat

In [ ]:
sorted_chats = sorted(chats.items(), key = lambda x: x[0])
sorted_chats

[(49, 'Did you change your hair?'),
 (50, 'No.'),
 (51, 'You might wanna think about it'),
 (59, 'I missed you.'),
 (60, 'It says here you exposed yourself to a group of freshmen girls.'),
 (61, 'It was a bratwurst.  I was eating lunch.'),
 (62, 'With the teeth of your zipper?'),
 (63, 'You the new guy?'),
 (64, 'So they tell me...'),
 (65, "C'mon.  I'm supposed to give you the tour."),
 (66, 'So -- which Dakota you from?'),
 (67, "North, actually.  How'd you   ?"),
 (68, 'I was kidding. People actually live there?'),
 (69, "Yeah.  A couple.  We're outnumbered by the cows, though."),
 (70, 'How many people were in your old school?'),
 (71, 'Thirty-two.'),
 (72, 'Get out!'),
 (73, 'How many people go here?'),
 (74, 'Couple thousand. Most of them evil'),
 (77, "That I'm used to."),
 (78,
  'Yeah, but these guys have never seen a horse.  They just jack off to Clint Eastwood.'),
 (87, 'That girl -- I --'),
 (88, 'You burn, you pine, you perish?'),
 (89, 'Who is she?'),
 (90, "Bianca Stratf

In [ ]:
conves_dict = {}
counter = 1
conves_ids = []
for i in range(1, len(sorted_chats)+1):
    if i < len(sorted_chats):
        if (sorted_chats[i][0] - sorted_chats[i-1][0]) == 1:
            if sorted_chats[i-1][1] not in conves_ids:
                conves_ids.append(sorted_chats[i-1][1])
            conves_ids.append(sorted_chats[i][1])
        elif (sorted_chats[i][0] - sorted_chats[i-1][0]) > 1:            
            conves_dict[counter] = conves_ids
            conves_ids = []
        counter += 1
    else:
        pass

In [ ]:
context_and_target = []
#dicarding dialogue without a reply
for conves in conves_dict.values():
    if len(conves) % 2 != 0:
        conves = conves[:-1]
    for i in range(0, len(conves), 2):
        context_and_target.append((conves[i], conves[i+1]))

In [ ]:
context, target = zip(*context_and_target)
context = list(context)
target = list(target)

In [ ]:
import re
def clean_text(text):
    # Clean text by removing unnecessary characters and altering the format of words

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [ ]:
tidy_context = []
for conve in context:
    text = clean_text(conve)
    tidy_context.append(text)
tidy_target = []
for conve in target:
    text = clean_text(conve)
    tidy_target.append(text)

In [ ]:
length_list=[]
for l in tidy_context:
    length_list.append(len(l.split(' ')))
for l in tidy_target:
    length_list.append(len(l.split(' ')))

In [ ]:
lengths = pd.DataFrame(length_list, columns=['counts'])
lengths['counts'].describe()

count    287728.000000
mean         11.307634
std          13.115849
min           1.000000
25%           4.000000
50%           7.000000
75%          14.000000
max        1128.000000
Name: counts, dtype: float64

In [ ]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
20.0
25.0
34.0
61.0


In [ ]:
len(tidy_context)==len(tidy_target)

True

In [ ]:
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(tidy_context):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(tidy_target[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(tidy_target):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(tidy_context[i])
        
print(len(short_questions))
print(len(short_answers))

113507
113507


In [ ]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

i mean a parking lot
yah well seven hundred and fifty thousand dollars is a lot  ha ha ha

well you know stan will say no dice that is why you pay him  i am asking you here wade this could work out real good for me and jean and scotty 
jean and scotty never have to worry

come on man okay here's an idea we will stop outside of brainerd i know a place there we can get laid wuddya think
i am fucking hungry now you know



In [ ]:
#choosing number of samples
num_samples = 30000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]


In [ ]:
#append start and end tokens for the answers
short_answers2 = []
for ans in short_answers:
    ans = '<SOS> ' + ans + ' <EOS>'
    short_answers2.append(ans)

In [ ]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers2:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [ ]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 20
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [ ]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 23909
Size of vocab we will use: 1788


In [ ]:
#we will create dictionaries to provide a unique integer for each word.
vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        vocab_to_int[word] = word_num
        word_num += 1

In [ ]:
len(vocab_to_int)

1788

In [ ]:
# Add the unique tokens (pad and unknown vocab) to the vocabulary dictionaries.
codes = ['<PAD>','<UNK>']
for code in codes:
    code_int = len(vocab_to_int)
    vocab_to_int[code] = code_int

In [ ]:
#switch <PAD> value to well's value of 0 for padding purposes later
print(vocab_to_int['<PAD>'])
for i, v in vocab_to_int.items():
    if v == 0:
        print(i)

1788
i


In [ ]:
for i, v in vocab_to_int.items():
    if v == 0:
        vocab_to_int[i] = vocab_to_int['<PAD>']
vocab_to_int['<PAD>'] = 0

In [ ]:
len(vocab_to_int)

1790

In [ ]:
faq=pd.read_csv('/content/drive/MyDrive/faq.csv')
faq.head()

,Question,Answer
0,Are session pre-class quizzes graded?,No. Pre-class quiz is to just check your under...
1,When is the deadline for the submission of ses...,5 PM on the day of the following lecture.
2,Will there be any extension allowed for the qu...,Only for exceptional cases.
3,How many hours will I need to dedicate to succ...,About 15 hours per week.
4,Who will grade my exercise?,The exercises are auto-graded once you click t...


In [ ]:
tidy_faq_ques = []
for conve in faq.Question:
    text = clean_text(conve)
    tidy_faq_ques.append(text)
tidy_faq_ans = []
for conve in faq.Answer:
    text = clean_text(conve)
    tidy_faq_ans.append(text)

In [ ]:
length_list_faq=[]
for l in tidy_faq_ques :
    length_list_faq.append(len(l.split(' ')))
for l in tidy_faq_ans:
    length_list_faq.append(len(l.split(' ')))
lengths_faq = pd.DataFrame(length_list_faq, columns=['counts'])
lengths_faq['counts'].describe()

count    66.000000
mean      9.878788
std       6.505655
min       2.000000
25%       6.250000
50%       9.000000
75%      12.000000
max      51.000000
Name: counts, dtype: float64

In [ ]:
print(np.percentile(lengths_faq, 80))
print(np.percentile(lengths_faq, 85))
print(np.percentile(lengths_faq, 90))
print(np.percentile(lengths_faq, 95))
print(np.percentile(lengths_faq, 99))

13.0
13.0
15.0
17.75
31.49999999999983


In [ ]:
vocabs = {}
for question in tidy_faq_ques:
    for word in question.split():
        if word not in vocabs:
            vocabs[word] = 1
        else:
            vocabs[word] += 1
            
for answer in tidy_faq_ans:
    for word in answer.split():
        if word not in vocabs:
            vocabs[word] = 1
        else:
            vocabs[word] += 1

In [ ]:
for i in vocab_to_int:
  if i in vocabs:
    vocabs[i]=0

In [ ]:
word_num=1790
for word, count in vocabs.items():
    if count > 0:
        vocab_to_int[word] = word_num
        word_num += 1

In [ ]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

In [ ]:
# Check the length of the dictionaries.
print(len(vocab_to_int))
print(len(int_to_vocab))

1911
1911


In [ ]:
tidy_faq_ans2 = []
for ans in tidy_faq_ans:
    ans = '<SOS> ' + ans + ' <EOS>'
    tidy_faq_ans2.append(ans)

In [ ]:
min_line_length = 2
max_line_length = 20

faq_ques_final = []
faq_ans_final = []
faq_ans2_final = []

for i, answer in enumerate(tidy_faq_ans):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        faq_ans_final.append(answer)
        faq_ques_final.append(tidy_faq_ques[i])
        faq_ans2_final.append(tidy_faq_ans2[i])

In [ ]:
print(len(faq_ques_final))
print(len(faq_ans_final))
print(len(faq_ans2_final))

31
31
31


In [ ]:
print(len(short_questions))
print(len(short_answers))
print(len(short_answers2))

30000
30000
30000


In [ ]:
# Convert the text to integers with paddings
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers2:
    ints = []
    for word in answer.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int.append(ints)
# Check the lengths
print(len(questions_int))
print(len(answers_int))

30000
30000


In [ ]:
# Convert the text to integers with paddings
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int_faq = []
for question in faq_ques_final:
    ints = []
    for word in question.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    questions_int_faq.append(ints)
    
answers_int_faq = []
for answer in faq_ans2_final:
    ints = []
    for word in answer.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    answers_int_faq.append(ints)
# Check the lengths
print(len(questions_int_faq))
print(len(answers_int_faq))

31
31


In [ ]:
#include padding
encoder_input_data = pad_sequences(questions_int, maxlen=max_line_length, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length
decoder_input_data = pad_sequences(answers_int, maxlen=max_line_length+2, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length + start and end tokens

In [ ]:
#include padding
encoder_input_data_faq = pad_sequences(questions_int_faq, maxlen=max_line_length, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length
decoder_input_data_faq = pad_sequences(answers_int_faq, maxlen=max_line_length+2, value=vocab_to_int['<PAD>'], padding='post') #pad to max_line_length + start and end tokens

In [ ]:
#decoder target is 1 timestep (word) ahead of decoder input, in a 3-d array
decoder_target_data = np.zeros(
    (len(answers_int), max_line_length+2, len(vocab_to_int)), #memory error occurs after 3500
    dtype='float32')

In [ ]:
#decoder target is 1 timestep (word) ahead of decoder input, in a 3-d array
decoder_target_data_faq = np.zeros(
    (len(answers_int_faq), max_line_length+2, len(vocab_to_int)), #memory error occurs after 3500
    dtype='float32')

In [ ]:
for i, target_seq in enumerate(answers_int):
    for t, seq in enumerate(target_seq):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, seq] = 1.

In [ ]:
for i, target_seq in enumerate(answers_int_faq):
    for t, seq in enumerate(target_seq):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data_faq[i, t - 1, seq] = 1.

In [ ]:
print (encoder_input_data.shape)
print (decoder_input_data.shape)
print (decoder_target_data.shape)

(30000, 20)
(30000, 22)
(30000, 22, 1911)


In [ ]:
print (encoder_input_data_faq.shape)
print (decoder_input_data_faq.shape)
print (decoder_target_data_faq.shape)

(31, 20)
(31, 22)
(31, 22, 1911)


In [ ]:
!unzip "/content/drive/MyDrive/glove.6B.50d.txt.zip" -d "/content/glove"

Archive:  /content/drive/MyDrive/glove.6B.50d.txt.zip
  inflating: /content/glove/glove.6B.50d.txt  


In [ ]:

embeddings_index = {}
with open('/content/glove/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

Glove Loded!


In [ ]:
embeddings_index['<SOS>']=np.random.rand(50)
embeddings_index['EOS']=np.random.rand(50)
embeddings_index['UNK']=np.random.rand(50)

In [ ]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
embedding_matrix = embedding_matrix_creater(embedding_dimention, vocab_to_int)

In [ ]:
embedding_matrix.shape

(1912, 50)

In [ ]:
embedding_size = 50
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(len(vocab_to_int)+1, embedding_size,weights=[embedding_matrix])(encoder_inputs)
encoder = Bidirectional(LSTM(100, return_state=True))
encoder_outputs, state_h_1, state_c_1, state_h_2, state_c_2 = encoder(en_x)

In [ ]:
state_h = concatenate([state_h_1, state_h_2], axis=1)
state_c = concatenate([state_c_1, state_c_1], axis=1)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
#non-bidirectional approach
# encoder_inputs = Input(shape=(None,))
# en_x=  Embedding(len(vocab_to_int)+1, embedding_size,weights=[embedding_matrix])(encoder_inputs)
# encoder = LSTM(50, return_state=True)
# encoder_outputs, state_h, state_c = encoder(en_x)
# # We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(len(vocab_to_int), embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(vocab_to_int), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) #sparse_categorical_crossentropy as labels in a single integer array|

In [ ]:
model.summary()

Model: "model_51"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 50)     95600       input_58[0][0]                   
__________________________________________________________________________________________________
input_59 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) [(None, 200), (None, 120800      embedding_13[0][0]               
___________________________________________________________________________________________

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=16,
          epochs=20,
          validation_split=0.05
         )

Epoch 1/20
1782/1782 [==============================] - 29s 14ms/step - loss: 2.1201 - acc: 0.0518 - val_loss: 2.0380 - val_acc: 0.0643
Epoch 2/20
1782/1782 [==============================] - 24s 13ms/step - loss: 1.9930 - acc: 0.0652 - val_loss: 1.8840 - val_acc: 0.0819
Epoch 3/20
1782/1782 [==============================] - 24s 13ms/step - loss: 1.8416 - acc: 0.0820 - val_loss: 1.8011 - val_acc: 0.0888
Epoch 4/20
1782/1782 [==============================] - 24s 13ms/step - loss: 1.7797 - acc: 0.0906 - val_loss: 1.7706 - val_acc: 0.0956
Epoch 5/20
1782/1782 [==============================] - 24s 14ms/step - loss: 1.7391 - acc: 0.0933 - val_loss: 1.7532 - val_acc: 0.0944
Epoch 6/20
1782/1782 [==============================] - 24s 14ms/step - loss: 1.7143 - acc: 0.0956 - val_loss: 1.7466 - val_acc: 0.0948
Epoch 7/20
1782/1782 [==============================] - 24s 14ms/step - loss: 1.7002 - acc: 0.0970 - val_loss: 1.7396 - val_acc: 0.0972
Epoch 8/20
1782/1782 [==========================

In [ ]:
model.save('/content/drive/MyDrive/final6_before_fine.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_52"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 50)     95600       input_58[0][0]                   
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) [(None, 200), (None, 120800      embedding_13[0][0]               
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 200)          0           bidirectional_6[0][1]            
                                                                 bidirectional_6[0][3]     

In [ ]:
#Create sampling model
decoder_state_input_h  = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['<SOS>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<EOS>' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for i in range(5):
    seq_index = np.random.randint(1, len(encoder_input_data))
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', short_answers[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['why me']
Decoded sentence:  i am sorry <EOS>
-
Input sentence: [' armed robbery ']
Decoded sentence:  <UNK> <UNK> <UNK> <EOS>
-
Input sentence: ['nobody else can']
Decoded sentence:  you are <UNK> <EOS>
-
Input sentence: ['does not look anything like her to me']
Decoded sentence:  i am sorry <EOS>
-
Input sentence: ['he is like the pope like we are gonna snake a girl away from the pope']
Decoded sentence:  i am sorry <EOS>


In [ ]:
model.load_weights('/content/drive/MyDrive/final6_before_fine.h5')

In [ ]:
model.fit([encoder_input_data_faq, decoder_input_data_faq], decoder_target_data_faq,
          batch_size=16,
          epochs=300,
          validation_split=0.05
         )

Epoch 1/300
2/2 [==============================] - 0s 60ms/step - loss: 4.5443 - acc: 0.0596 - val_loss: 2.8912 - val_acc: 0.0455
Epoch 2/300
2/2 [==============================] - 0s 34ms/step - loss: 3.5686 - acc: 0.0878 - val_loss: 2.7095 - val_acc: 0.0455
Epoch 3/300
2/2 [==============================] - 0s 32ms/step - loss: 3.1137 - acc: 0.0925 - val_loss: 2.6220 - val_acc: 0.0455
Epoch 4/300
2/2 [==============================] - 0s 34ms/step - loss: 2.8353 - acc: 0.0972 - val_loss: 2.5520 - val_acc: 0.0455
Epoch 5/300
2/2 [==============================] - 0s 36ms/step - loss: 2.6783 - acc: 0.1034 - val_loss: 2.5805 - val_acc: 0.0455
Epoch 6/300
2/2 [==============================] - 0s 37ms/step - loss: 2.5609 - acc: 0.1034 - val_loss: 2.4990 - val_acc: 0.0455
Epoch 7/300
2/2 [==============================] - 0s 34ms/step - loss: 2.4654 - acc: 0.1097 - val_loss: 2.4952 - val_acc: 0.0455
Epoch 8/300
2/2 [==============================] - 0s 33ms/step - loss: 2.3816 - acc: 0.11

In [ ]:
model.save('/content/drive/MyDrive/final6_after_fine.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_54"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 50)     95600       input_58[0][0]                   
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) [(None, 200), (None, 120800      embedding_13[0][0]               
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 200)          0           bidirectional_6[0][1]            
                                                                 bidirectional_6[0][3]     

In [ ]:
#Create sampling model
decoder_state_input_h  = Input(shape=(200,))
decoder_state_input_c = Input(shape=(200,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence_faq(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((20,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['<SOS>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = int_to_vocab[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<EOS>' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((20,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for i in range(10):
  seq_index = np.random.randint(1, len(encoder_input_data_faq))
  # Take one sequence (part of the training set)
  # for trying out decoding.
  input_seq = encoder_input_data_faq[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', faq_ques_final[seq_index: seq_index + 1])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['when will homework be released ']
Decoded sentence:  the homework release is subject to change and hence will
-
Input sentence: ['where do i find the recording ']
Decoded sentence:  recording links will be posted on edstem <EOS>
-
Input sentence: ['i have a different approach to the exercise why is the autograder still failing me']
Decoded sentence:  unfortunately the autograder does not accept alternative
-
Input sentence: ['what is the link for the slack channel']
Decoded sentence:  all official communication by prof protopapas will be
-
Input sentence: ['will there be any extension allowed for the quiz and exercise submission']
Decoded sentence:  only for exceptional cases <EOS>
-
Input sentence: ['where do i find the recording ']
Decoded sentence:  recording links will be posted on edstem <EOS>
-
Input sentence: ['when will the reading be published ']
Decoded sentence:  after every lab we will post the reading for the next
-
Input sentence: ['what is the zoom th

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/test.csv')
test.head()

,Questions
0,Will the pre-class session be recorded?
1,What is the deadline for quiz submission?
2,What is the deadline for exercise submission?
3,How many hours do I need to complete this course?
4,Who will grade the exercise?


In [ ]:
tidy_test_ques = []
for conve in test.Questions:
    text = clean_text(conve)
    tidy_test_ques.append(text)

In [ ]:
test_ques=[]
for i, question in enumerate(tidy_test_ques):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        test_ques.append(question)

In [ ]:
ques_int_test = []
for question in test_ques:
    ints = []
    for word in question.split():
        if word not in vocab_to_int:
            ints.append(vocab_to_int['<UNK>'])
        else:
            ints.append(vocab_to_int[word])
    ques_int_test.append(ints)

In [ ]:
encoder_input_data_test = pad_sequences(ques_int_test, maxlen=max_line_length, value=vocab_to_int['<PAD>'], padding='post')

In [ ]:
preds=[]
for i in encoder_input_data_test:
  input_seq=i
  decoded_sentence=decode_sequence_faq(input_seq)
  preds.append(decoded_sentence)

In [ ]:
test['Anwers']=preds
test

,Questions,Anwers
0,Will the pre-class session be recorded?,the <UNK> is a violation of our code conduct ...
1,What is the deadline for quiz submission?,least sir the hell posted on edstem <EOS>
2,What is the deadline for exercise submission?,least sir the hell posted on edstem <EOS>
3,How many hours do I need to complete this course?,the long comes <EOS>
4,Who will grade the exercise?,you is your violation of the reading it will ...
5,Why is the auto-grader failing me?,i will be posted on edstem <EOS>
6,Do I do the exercises individually?,i will be posted on edstem <EOS>
7,Is the lab compulsory?,least a violation of our code conduct <EOS>
8,Will the sessions be recorded?,the <UNK> is a violation of our code conduct ...
9,Can I have access to the recorded videos?,the homework release is subject to change and...
